# Обработка датасета

- rename_on_order_in_folder - Переименовать изображения и метки по порядку


- ImageLabelViewer - класс для просмотра изображений (run_viewer() - запуск)


# Библиотеки

In [2]:
import os
import cv2

from Module import rename_on_order_in_folder
from Module import ImageLabelViewer, DatasetCreate

# Константы

In [3]:
FOLDER_WINTER_PATH = "./DataSet/MyData/Winter/"
FOLDER_SUMMER_PATH = "./DataSet/MyData/Summer/"

FOLDER_TRAIN_PATH = "./DataSet/SplitData/Train2/"

FOLDER_VALID_PATH = "./DataSet/MyData/Valid/"
FOLDER_TEST_PATH = "./DataSet/MyData/Test/"

Переименовать файлы в папке поочереди

In [4]:
# rename_on_order_in_folder(FOLDER_SUMMER_PATH, nweName="")

Расформировать данные train valid test

In [5]:
# new_data = DatasetCreate(
#     FOLDER_WINTER_PATH, 
#     FOLDER_SUMMER_PATH, 
#     FOLDER_TRAIN_PATH, 
#     FOLDER_VALID_PATH, 
#     FOLDER_TEST_PATH)

# new_data.create_data_for_dataset()


Просмотр картинко с рамками в папке

In [6]:
# использование класса
viewer = ImageLabelViewer(FOLDER_TRAIN_PATH)
viewer.run_viewer()